In [ ]:
import sys
sys.path.insert(0, '/Users/timkrausz/Documents/GitHub/DA_maze_paper/DA/Modules/')

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()
plt.style.use('default')
plt.style.use("seaborn-poster")
from multi_rat_da import *
from hexLevelAnalyses import *
from photometryQuantifications import *
from valPropRats import *
from photometrySessionVizualizations import *
%matplotlib qt

# **DF columns:** 
#### *green_z_scored*: isosbestic-corrected z-scored dLight signal
#### *port*: port ID (0,1,2 for A,B,C) at moment of first entry per trial and -100 otherwise
#### *hexlabel*: hex ID (1-49)
#### *rwd*: reward received (0,1)
#### *session*: session identifier number
#### *block*: block ID
#### *tri*: trial number within block
#### *rat*
#### *date*
#### *session_type*: barrier or probability variant
#### *vel*: velocity
#### *lenAB*, *lenBC*, *lenAC* path length (in hexes) between labeled ports
#### *nom_rwd_a*, *nom_rwd_b*, *nom_rwd_c*: nominal reward probability for ports
#### *dtop*: distance (in hexes) to the destination port
#### *lastleave*: 1 marks time when rat last left one port before entering another. 2 marks times after leaving port. 0 otherwise.
#### *beamA, beamB, beamC*: 1 where infrared beam was broken at port, 0 otherwise.
#### *pairedHexState*: hex-state defined based on direction of entry (function of prior hexlabel)

# Load dataframe

In [3]:
loadpath = "/Volumes/Tim/Photometry/10MfRatDataSet/data4sharing/"
datName = "photLevelDf"
df = pd.read_csv(loadpath+datName+".csv")
photrats = PhotRats(None)
photrats.df = reduce_mem_usage(df)
del df
photrats.directory_prefix = loadpath

# compute nom_rwd_chosen and add to dataframe

In [6]:
photrats.factor = "nom_rwd"
photrats.get_vals_byChosenEtc(chosen_only=True)

In [10]:
create_triframe(photrats)

## add port q values to dataframe

In [11]:
photrats.get_portQvals(qtype="port",level="rat")

## plot individual session dLight, reference, and velocity traces

In [19]:
plt.figure()
plt.hist(photrats.df.loc[photrats.df.session==25,"green_z_scored"].values,bins=100)
plt.tight_layout()

In [238]:
photrats.directory_prefix = '/Volumes/Tim-1/Photometry Data/'

In [241]:
fig,s = plot_individualSeshTrace_RwdAndNewHexLabeled(photrats,\
                                        75,plot_newlyAvail=False,plot_ref=True)

In [240]:
fig.savefig(loadpath+"s"+str(s)+"_individual_sesh_trace_rampRwdAndNewlyAvailEntry.pdf")

In [243]:
fig.savefig(loadpath+"s"+str(s)+"_individual_sesh_trace_rampAndRwd.pdf")

## plot peak/trough after port entry as a function of p(rwd)

In [9]:
photrats.set_plot_trace("green_z_scored")

In [7]:
photrats.pool_factor = "nom_rwd_chosen"

#rewarded trials
photrats.dat = photrats.df.loc[(photrats.df.rwd==1)&(photrats.df.tri>25),]
lowIndsRwd,midIndsRwd,highIndsRwd = photrats.getTriIndsByTerc(rwdtype="rwd")

#omission trials
photrats.dat = photrats.df.loc[(photrats.df.rwd!=1)&(photrats.df.tri>25),]
lowIndsOm,midIndsOm,highIndsOm = photrats.getTriIndsByTerc(rwdtype="om")

In [10]:
fig = plt.figure(figsize=(4,5.5))
plot_peakTroughDaDifAfterPortEntry_barWithRats(photrats,highIndsRwd,midIndsRwd,lowIndsRwd)
plot_peakTroughDaDifAfterPortEntry_barWithRats(photrats,highIndsOm,midIndsOm,lowIndsOm,peak=False,pltCol="blue")

In [16]:
rwdCors,omCors = calc_DaChangeVprobCors(photrats)
plt.suptitle("rwd pval = "+str(wilcoxon(np.array(rwdCors)[:,0])[1])+\
            "\n om pval = "+str(wilcoxon(np.array(omCors)[:,0])[1]))
fig.savefig(photrats.directory_prefix+"pRwdDAPeak_Trough_Dif_RPE_BarPlot_correctAlignment.pdf")

In [ ]:
rwdCors = pd.read_csv(photrats.directory_prefix+"pearsonR_result_DaVsRpe_rwd_correctAlignment.csv")
omCors = pd.read_csv(photrats.directory_prefix+"pearsonR_result_DaVsRpe_om_correctAlignment.csv")

In [ ]:
omCors.coef

In [17]:
wilcoxon(rwdCors.coef,alternative="less")

AttributeError: 'list' object has no attribute 'coef'

In [ ]:
wilcoxon(omCors.coef,alternative="less")

- get new vs blocked hex entry info
  - first need to load transition matrices and barrier configs

In [29]:
tmat_path = ""#directory of transition matrices folder
photrats.directory_prefix = tmat_path
photrats.load_tmats()
photrats.get_barIDs()
photrats.directory_prefix = loadpath#photrats.directory_prefix+"

no tmat saved for sesh 0 block 7
no tmat saved for sesh 45 block 4
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
directory_prefix back to nas 8
briefly switching directory_prefix to new Nas...
dire

- plot port-aligned DA pooled by p(rwd) second half of blocks

In [27]:
create_triframe(photrats)

In [28]:
photrats.triframe.to_csv(photrats.directory_prefix+"triframe.csv")

In [20]:
photrats.get_portQvals(qtype="port",level="rat")
#get_portQvals(photrats,qtype="port",level="rat")

In [21]:
photrats.factor = "Q"
photrats.get_vals_byChosenEtc(chosen_only=True)

In [23]:
photrats.set_plot_window([-5,5])
fig = plot_portAlignedDaInTime(photrats,secondHalfOnly=False,poolFactor="Q_chosen",useRatGroupLevel=True)
plt.xticks(np.arange(-5,6),np.arange(-5,6))
plt.ylim(-.9,2.2)

/Users/timkrausz/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maze-photometry-analyses/photometryQuantifications.py:124: RuntimeWarning: Mean of empty slice
  ax.plot(xvals[subset[0]:subset[1]],np.nanmean\
/Users/timkrausz/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maze-photometry-analyses/photometryQuantifications.py:126: RuntimeWarning: Mean of empty slice
  ax.fill_between(xvals[subset[0]:subset[1]],(np.nanmean\
/Users/timkrausz/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maze-photometry-analyses/photometryQuantifications.py:128: RuntimeWarning: Mean of empty slice
  [subset[0]:subset[1]],(np.nanmean\
/Users/timkrausz/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maze-photometry-analyses/photometryQuantifications.py:124: RuntimeWarning: Mean of empty slice
  ax.plot(xvals[subset[0]:subset[1]],np.nanmean\
/Users/timkrausz/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maze-photometry-analyses/photometryQuantifications.py:126: RuntimeW

(-0.9, 2.2)

In [24]:
fig.savefig(photrats.directory_prefix+"da_rpe_by_portQ_ratAvg_correctAlignment.pdf")

In [25]:
photrats.set_plot_window([-5,5])
fig = plot_portAlignedDaInTime(photrats,secondHalfOnly=True,poolFactor="nom_rwd_chosen",useRatGroupLevel=True)
plt.xticks(np.arange(-5,6),np.arange(-5,6))
plt.ylim(-.9,2.2)

/Users/timkrausz/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maze-photometry-analyses/photometryQuantifications.py:124: RuntimeWarning: Mean of empty slice
  ax.plot(xvals[subset[0]:subset[1]],np.nanmean\
/Users/timkrausz/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maze-photometry-analyses/photometryQuantifications.py:126: RuntimeWarning: Mean of empty slice
  ax.fill_between(xvals[subset[0]:subset[1]],(np.nanmean\
/Users/timkrausz/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maze-photometry-analyses/photometryQuantifications.py:128: RuntimeWarning: Mean of empty slice
  [subset[0]:subset[1]],(np.nanmean\
/Users/timkrausz/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maze-photometry-analyses/photometryQuantifications.py:124: RuntimeWarning: Mean of empty slice
  ax.plot(xvals[subset[0]:subset[1]],np.nanmean\
/Users/timkrausz/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maze-photometry-analyses/photometryQuantifications.py:126: RuntimeW

(-0.9, 2.2)

In [26]:
fig.savefig(photrats.directory_prefix+"da_rpe_byportQ_2ndHalf_ratAvg_correctAlignment.pdf")

## plot q rpe lag reg

In [65]:
photrats.bin_size = int(photrats.fs/15)
ratRwdRpes,ratOmRpes,ratRwdNs,ratOmNs = calcRpeRegByRatAndSesh(photrats,useQ=True)

100%|██████████| 10/10 [05:43<00:00, 34.36s/it]


In [83]:
plot_rpeLagRegCoefs(photrats,binsize=1000/12.5)

In [38]:
regWeightsByRat = calc_choiceRegWeightsByRat(photrats)

In [18]:
np.array(photrats.triframe.rat.unique()).astype(str)

array(['IM-1272', 'IM-1273', 'IM-1276', 'IM-1291', 'IM-1292', 'IM-1322',
       'IM-1398', 'IM-1434', 'IM-1458', 'IM-1478'], dtype='<U7')

In [39]:
regWeightsByRat.loc[:,"rat"] = np.array(photrats.triframe.rat.unique())
regWeightsByRat

,intercept,relative p(R),relative distance,rat
0,0.788138,0.840152,-2.901712,IM-1272
1,-0.067526,0.819950,-1.460891,IM-1273
2,0.703347,0.881085,-2.522147,IM-1276
3,0.030442,0.549869,-1.716633,IM-1291
4,0.166691,1.393542,-1.911691,IM-1292
5,0.668904,1.152568,-2.430597,IM-1322
6,0.865570,0.666288,-2.379002,IM-1398
7,-0.615642,1.637882,-1.195741,IM-1434
8,-0.605195,1.244594,-0.726433,IM-1478


- plot difference between peak DA within 0.5s after port entry and DA at port entry.
  - first take average trace for each rat and THEN take difference, not average of differences

- plot DA aligned to new/blocked hex discovery
  - identify indices of blocked and new path discovery
  - identify indices of new path before enter and new path before ignored

In [32]:
portstrings = ['A','B','C']
newDistsToPort = []
oldDistsToPort = []
for i in range(len(newHexAdjInds)):
    if newHexAdjStates[i] == -1:
        newDistsToPort.append(np.nan)
        oldDistsToPort.append(np.nan)
        continue
    state = newHexAdjStates[i]
    sesh = photrats.df.loc[newHexAdjInds[i],'session']
    block = photrats.df.loc[newHexAdjInds[i],'block']
    chosenPort = photrats.df.port.replace(-100,method='bfill')[newHexAdjInds[i]]#photrats.df.loc[newHexAdjInds[i],'']
    newDistsToPort.append(photrats.sesh_hexDists['dto'+portstrings[chosenPort]][sesh][block-1][state])
    oldDistsToPort.append(photrats.sesh_hexDists['dto'+portstrings[chosenPort]][sesh][block-2][state])
newDistsToPort = np.array(newDistsToPort)
oldDistsToPort = np.array(oldDistsToPort)

In [33]:
#drop all entries where newHexAdjStates==-1
newHexAdjStates = np.array(newHexAdjStates)
newHexAdjInds = np.array(newHexAdjInds)
toRemove = np.where(newHexAdjStates==-1)[0]
newHexAdjStates = np.delete(newHexAdjStates,toRemove)
newHexAdjInds = np.delete(newHexAdjInds,toRemove)
newDistsToPort = np.delete(newDistsToPort,toRemove)
oldDistsToPort = np.delete(oldDistsToPort,toRemove)

In [71]:
newDistsToPort

array([ 4., 10.,  9.,  7., 15., 16.,  9.,  8.,  9.,  9., 17.,  7.,  4.,
        7.,  4.,  9.,  8.,  6., 11.,  8.,  8.,  4., 15.,  9.,  8.,  5.,
        8.,  9.,  9.,  7.,  9.,  8., 10., nan,  7.,  7.,  9., nan,  5.,
       15.,  4., 14.,  8.,  8., 10.,  4.,  7.,  8., 10., 12.,  7., 11.,
        9.,  7., 15., 16., 12.,  8.,  4., 10., 18.,  7., 11.,  7., 10.,
        8.,  6.,  5., 10.,  7., 11.,  7.,  9.,  6.,  7.,  4.,  4.,  6.,
        9.,  6.,  7.,  6.,  7., 18.,  7.,  8.,  5., 10.,  9., 13., 10.,
        8.,  9., 10.,  7.,  6.,  9.,  7., 12., 11.,  7.,  9.,  5., 13.])

In [73]:
relDist2Port = newDistsToPort-oldDistsToPort

In [74]:
adjHexIndsSortedByRelDist = newHexAdjInds[relDist2Port.argsort()]
#relDist2Port = relDist2Port[relDist2Port.argsort()]
#toRemove = np.where(np.isnan(relDist2Port))[0]
#adjHexIndsSortedByRelDist = np.delete(adjHexIndsSortedByRelDist,toRemove)
#relDist2Port = np.delete(relDist2Port,toRemove)

In [34]:
adjHexIndsSortedByDist = newHexAdjInds[newDistsToPort.argsort()]
#newDistsToPort = newDistsToPort[newDistsToPort.argsort()]
#toRemove = np.where(np.isnan(newDistsToPort))[0]
#adjHexIndsSortedByDist = np.delete(adjHexIndsSortedByDist,toRemove)
#newDistsToPort = np.delete(newDistsToPort,toRemove)

In [35]:
photrats.set_plot_trace("green_z_scored")
photrats.set_plot_window([-5,5])

In [36]:
adjHexNextPortVal = photrats.df.loc[newHexAdjInds,"nom_rwd_chosen"].values
adjHexIndsSortedByPortVal = newHexAdjInds[adjHexNextPortVal.argsort()]
#adjHexNextPortVal = adjHexNextPortVal[adjHexNextPortVal.argsort()]

### now do it by rat

In [63]:
newHexAdjStates = np.delete(newHexAdjStates,toRemove)
newHexAdjInds = np.delete(newHexAdjInds,toRemove)

In [38]:
ratNewPathLens = {r:[] for r in photrats.df.rat.unique()}
for rat in photrats.df.rat.unique():
    ratNewPathLens[rat] = newDistsToPort[np.where(np.isin(newHexAdjInds,photrats.df.loc[photrats.df.rat==rat].index))[0]]
print(ratNewPathLens)


{'IM-1272': array([ 4., 10.,  9.,  7., 15., 16.,  9.,  8.,  9.,  9., 17.,  7.,  4.,
        7.,  4.,  9.,  8.,  6.]), 'IM-1273': array([11.,  8.,  8.]), 'IM-1276': array([ 4., 15.,  9.,  8.,  5.,  8.,  9.,  9.,  7.,  9.,  8., 10., nan,
        7.,  7.,  9., nan,  5.]), 'IM-1291': array([15.,  4., 14.,  8.,  8., 10.,  4.,  7.,  8.]), 'IM-1292': array([10., 12.,  7., 11.,  9.,  7., 15., 16., 12.,  8.,  4., 10., 18.,
        7.]), 'IM-1322': array([11.,  7., 10.,  8.,  6.,  5., 10.,  7., 11.,  7.,  9.,  6.,  7.,
        4.,  4.]), 'IM-1398': array([ 6.,  9.,  6.,  7.,  6.,  7., 18.,  7.]), 'IM-1434': array([ 8.,  5., 10.,  9., 13., 10.,  8.]), 'IM-1478': array([ 9., 10.,  7.,  6.,  9.,  7., 12., 11.,  7.,  9.]), 'IM-1532': array([ 5., 13., 13., 12.])}


In [69]:
with open(loadpath+"lenFromNewHexNumsByRat.txt", 'w') as f:
    for key, value in ratNewPathLens.items():
        f.write(f'{key}: {value}\n')

In [161]:
ratsWithSufficientNumbers = ['IM-1272','IM-1276','IM-1291',\
            'IM-1292','IM-1322','IM-1398','IM-1434','IM-1478']

In [165]:
len(ratNewHexAdjInds)

10

In [164]:
n_plots = len(ratsWithSufficientNumbers)
n_cols = int(np.ceil(np.sqrt(n_plots)))
n_rows = int(np.ceil(n_plots / n_cols))
dist_cut = 6
use_relativeDist = False

fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(16,10))

xvals = np.arange(photrats.plot_window[0]*photrats.fs,photrats.plot_window[1]*photrats.fs+1)/photrats.fs

for i, ax in enumerate(axes.flatten()):
    if i < n_plots:
        rat = ratsWithSufficientNumbers[i]
        ratInds = np.where(np.isin(newHexAdjInds,photrats.df.loc[photrats.df.rat==rat].index))[0]
        ratNewHexAdjInds = newHexAdjInds[ratInds]
        ratNewHexDists = relDist2Port[ratInds] if use_relativeDist else newDistsToPort[ratInds]
        sortedRatInds = ratNewHexAdjInds[ratNewHexDists.argsort()]
        shortTrace,midTrace,longTrace = get_newPathTracesByDistToPort(sortedRatInds)
        #shortTrace,midTrace,longTrace = get_newPathTracesByDistToPort_absoluteDist(\
        #                        sortedRatInds,ratNewHexDists,dist_cutoff=dist_cut)
        ax.set_title(rat+"; "+str(len(shortTrace))+" short; "+str(len(midTrace))+" mid; "\
                    +str(len(longTrace))+" long")
        if len(shortTrace)>0:
            ax.plot(xvals,np.mean(shortTrace,axis=0),color='hotpink',label='short')
        if len(midTrace)>0:
            ax.plot(xvals,np.mean(midTrace,axis=0),color='red',label='mid')
        if len(longTrace)>0:
            ax.plot(xvals,np.mean(longTrace,axis=0),color='darkred',label='long')
        ax.axvline(x=0,ls='--',color='k',alpha=.8,lw=1)
        if (i + n_cols) >= n_plots:# i >= n_cols * (n_rows - 1):
            ax.set_xticks(np.arange(-5,6))
            ax.set_xlabel('time from hex entry (s)')
        else:
            ax.set_xticks([])
        # Only show the y axis if the subplot is in the first column
        if i % n_cols == 0:
            ax.set_ylabel('Mean DA (Z)')
        if i==0:
            ax.legend()
    else:
        # Remove any extra axes that are not needed
        ax.remove()

In [96]:
#fig.savefig(loadpath+"newPathDAbyDistToPort_byRat_"+str(dist_cut)+"hexGroups.pdf")
fig.savefig(loadpath+"newPathDAbyDistToPort_byRat_distTercile.pdf")

In [130]:
ratShortTraces = []
ratMidTraces = []
ratLongTraces = []
for rat in photrats.df.rat.unique():
    if rat == "IM-1273":
        continue
    ratInds = np.where(np.isin(newHexAdjInds,photrats.df.loc[photrats.df.rat==rat].index))[0]
    ratNewHexAdjInds = newHexAdjInds[ratInds]
    ratNewHexDists = newDistsToPort[ratInds]
    sortedRatInds = ratNewHexAdjInds[ratNewHexDists.argsort()]
    shortTrace,midTrace,longTrace = get_newPathTracesByDistToPort_absoluteDist(sortedRatInds,ratNewHexDists,dist_cutoff=7)
    ratShortTraces.append(np.mean(shortTrace,axis=0))
    ratMidTraces.append(np.mean(midTrace,axis=0))
    ratLongTraces.append(np.mean(longTrace,axis=0))

In [ ]:
plt.figure()
xvals = np.arange(photrats.plot_window[0]*photrats.fs,photrats.plot_window[1]*photrats.fs+1)/photrats.fs
toplt = np.mean(ratShortTraces,axis=0)
plt.plot(xvals,toplt,color='hotpink',label='short')
plt.fill_between(xvals,toplt+sem(ratShortTraces),toplt-sem(ratShortTraces),color='hotpink',alpha=0.5)
toplt = np.mean(ratMidTraces,axis=0)
plt.plot(xvals,toplt,color='red',label='mid')
plt.fill_between(xvals,toplt+sem(ratMidTraces),toplt-sem(ratMidTraces),color='red',alpha=0.5)
toplt = np.mean(ratLongTraces,axis=0)
plt.plot(xvals,toplt,color='darkred',label='long')
plt.fill_between(xvals,toplt+sem(ratLongTraces),toplt-sem(ratLongTraces),color='darkred',alpha=0.5)
plt.axvline(x=0,ls='--',color='k',alpha=.8,lw=1)
plt.xticks(np.arange(-5,6))
plt.legend()
plt.xlabel("time from hex entry (s)",fontsize="xx-large")
plt.ylabel("Mean DA (Z)",fontsize="xx-large")
plt.title("Average of rat averages")
plt.tight_layout()

In [66]:
%matplotlib qt

## plot newly available by distance to goal port from the current hex
- make sure I have shortestDistToPort for each block
- for each adjacent hex entered
  - identify the port the rat ran to
  - identify shortest distance to that port from the adjacent hex
  - save the length paired with the adjacent hex index in an array
  - identify the prior distance to the chosen port from the adjacent hex (block b-1) and save in array
  

In [43]:
shortTrace,midTrace,longTrace = get_newPathTracesByDistToPort(adjHexIndsSortedByDist)#,relDist2Port,dist_cut)
fig = plt.figure(figsize=(7,6))
xvals = np.arange(photrats.plot_window[0]*photrats.fs,photrats.plot_window[1]*photrats.fs+1)/photrats.fs
toplt = np.mean(shortTrace,axis=0)
plt.plot(xvals,toplt,color='hotpink',label='short')
plt.fill_between(xvals,toplt+sem(shortTrace),toplt-sem(shortTrace),color='hotpink',alpha=0.5)
toplt = np.mean(midTrace,axis=0)
plt.plot(xvals,toplt,color='red',label='mid')
plt.fill_between(xvals,toplt+sem(midTrace),toplt-sem(midTrace),color='red',alpha=0.5)
toplt = np.mean(longTrace,axis=0)
plt.plot(xvals,toplt,color='darkred',label='long')
plt.fill_between(xvals,toplt+sem(longTrace),toplt-sem(longTrace),color='darkred',alpha=0.5)
plt.axvline(x=0,ls='--',color='k',alpha=.8,lw=1)
plt.xticks(np.arange(-5,6))
plt.legend()
plt.xlabel("time from hex entry (s)",fontsize="xx-large")
plt.ylabel("Mean DA (Z)",fontsize="xx-large")
plt.title("Sorted by hex distance tercile\nAll rats pooled together.")
plt.ylim(-.6,2.8)
plt.tight_layout()
fig.savefig(photrats.directory_prefix+"newPathDAbyDistToPort_allRatsPooled_byTercile.pdf")

In [87]:
dist_cut = 7
shortTrace,midTrace,longTrace = get_newPathTracesByDistToPort_absoluteDist(newHexAdjInds,\
                                                                           newDistsToPort,dist_cut)
fig = plt.figure(figsize=(7,6))
xvals = np.arange(photrats.plot_window[0]*photrats.fs,photrats.plot_window[1]*photrats.fs+1)/photrats.fs
toplt = np.mean(shortTrace,axis=0)
plt.plot(xvals,toplt,color='hotpink',label='short')
plt.fill_between(xvals,toplt+sem(shortTrace),toplt-sem(shortTrace),color='hotpink',alpha=0.5)
toplt = np.mean(midTrace,axis=0)
plt.plot(xvals,toplt,color='red',label='mid')
plt.fill_between(xvals,toplt+sem(midTrace),toplt-sem(midTrace),color='red',alpha=0.5)
toplt = np.mean(longTrace,axis=0)
plt.plot(xvals,toplt,color='darkred',label='long')
plt.fill_between(xvals,toplt+sem(longTrace),toplt-sem(longTrace),color='darkred',alpha=0.5)
plt.axvline(x=0,ls='--',color='k',alpha=.8,lw=1)
plt.xticks(np.arange(-5,6))
plt.legend()
plt.xlabel("time from hex entry (s)",fontsize="xx-large")
plt.ylabel("Mean DA (Z)",fontsize="xx-large")
plt.title("Sorted by hex distance (groups of 6) from port\nAll rats pooled together.")
plt.tight_layout()
fig.savefig(photrats.directory_prefix+"newPathDAbyDistToPort_allRatsPooled_"+str(dist_cut)+"hexGroups.pdf")

In [171]:
adjHexNextPortVal[adjHexNextPortVal.argsort()]

array([10., 10., 10., 10., 10., 10., 10., 10., 20., 20., 20., 20., 20.,
       20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.,
       50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50.,
       50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50.,
       50., 50., 50., 50., 50., 50., 50., 50., 50., 80., 80., 80., 80.,
       80., 80., 80., 80., 80., 80., 80., 80., 80., 80., 80., 80., 80.,
       80., 80., 80., 80., 80., 80., 80., 80., 80., 80., 80., 80., 80.,
       80., 80., 80., 90., 90., 90., 90., 90., 90., 90., 90., 90., 90.],
      dtype=float16)

In [42]:

rwdCut=30
shortTrace,midTrace,longTrace = get_newPathTracesByDistToPort_absoluteDist(adjHexIndsSortedByPortVal,\
                                        adjHexNextPortVal[adjHexNextPortVal.argsort()],rwdCut)
fig = plt.figure(figsize=(7,6))
xvals = np.arange(photrats.plot_window[0]*photrats.fs,photrats.plot_window[1]*photrats.fs+1)/photrats.fs
toplt = np.mean(shortTrace,axis=0)
plt.plot(xvals,toplt,color='darkred',label='low')
plt.fill_between(xvals,toplt+sem(shortTrace),toplt-sem(shortTrace),color='darkred',alpha=0.5)
toplt = np.mean(midTrace,axis=0)
plt.plot(xvals,toplt,color='red',label='mid')
plt.fill_between(xvals,toplt+sem(midTrace),toplt-sem(midTrace),color='red',alpha=0.5)
toplt = np.mean(longTrace,axis=0)
plt.plot(xvals,toplt,color='hotpink',label='high')
plt.fill_between(xvals,toplt+sem(longTrace),toplt-sem(longTrace),color='hotpink',alpha=0.5)
plt.axvline(x=0,ls='--',color='k',alpha=.8,lw=1)
plt.xticks(np.arange(-5,6))
plt.legend()
plt.xlabel("time from hex entry (s)",fontsize="xx-large")
plt.ylabel("Mean DA (Z)",fontsize="xx-large")
plt.title("Sorted by port value tercile\nAll rats pooled together.")
plt.tight_layout()
fig.savefig(photrats.directory_prefix+"newPathDAbyPortVal_allRatsPooled_byCutoff.pdf")

In [40]:

shortTrace,midTrace,longTrace = get_newPathTracesByDistToPort(adjHexIndsSortedByPortVal)
fig = plt.figure(figsize=(7,6))
xvals = np.arange(photrats.plot_window[0]*photrats.fs,photrats.plot_window[1]*photrats.fs+1)/photrats.fs
toplt = np.mean(shortTrace,axis=0)
plt.plot(xvals,toplt,color='darkred',label='low')
plt.fill_between(xvals,toplt+sem(shortTrace),toplt-sem(shortTrace),color='darkred',alpha=0.5)
toplt = np.mean(midTrace,axis=0)
plt.plot(xvals,toplt,color='red',label='mid')
plt.fill_between(xvals,toplt+sem(midTrace),toplt-sem(midTrace),color='red',alpha=0.5)
toplt = np.mean(longTrace,axis=0)
plt.plot(xvals,toplt,color='hotpink',label='high')
plt.fill_between(xvals,toplt+sem(longTrace),toplt-sem(longTrace),color='hotpink',alpha=0.5)
plt.axvline(x=0,ls='--',color='k',alpha=.8,lw=1)
plt.xticks(np.arange(-5,6))
plt.legend()
plt.xlabel("time from hex entry (s)",fontsize="xx-large")
plt.ylabel("Mean DA (Z)",fontsize="xx-large")
plt.title("Sorted by port value tercile\nAll rats pooled together.")
plt.tight_layout()
plt.ylim(-.9,3.0)
fig.savefig(photrats.directory_prefix+"newPathDAbyPortVal_allRatsPooled_byTercile.pdf")

## should also plot by the p(rwd) of the subsequently entered port. And the p(rwd) of the port that hex is now closest to - the p(rwd) of the port the hex used to be closest to

In [30]:
photrats.get_newlyAvailHexesBySesh()
photrats.get_newlyBlockedHexesBySesh()
#add entries to df
photrats.add_newlyAvailHexesToDf()
photrats.add_adjacent2newlyAvail()
photrats.add_adjacent2newlyBlocked()

newHexInds,newHexStates,adjHexStates,adjHexInds = find_newHexEntryAndPriorHexInds(photrats)
blockedHexAdjInds,blockedHexAdjStates = find_blockedHexAdjInds(photrats)

blockedHexAdjInds = np.array(blockedHexAdjInds)[np.where(~np.isnan(blockedHexAdjInds))]
adjHexInds = np.array(adjHexInds)[np.where(~np.isnan(adjHexInds))]
adjHexInds = adjHexInds[adjHexInds!=-1]
blockedHexAdjInds = blockedHexAdjInds[blockedHexAdjInds!=-1]

newHexAdjInds,newHexAdjStates,enteredHex,enteredHexSoon = find_newHexAdjInds(photrats)
enteredInds,ignoredInds = find_enteredVignoredNewlyAvailInds(newHexAdjInds,enteredHex)
enteredInds = np.array(enteredInds)
ignoredInds = np.array(ignoredInds)

No previous adjacent hex entry detected for session  0  block  2
session  1  block  5
session  28  block  3
session  22  block  3
No previous adjacent hex entry detected for session  32  block  1
session  57  block  3
session  59  block  3
session  61  block  2
No previous adjacent hex entry detected for session  64  block  1
No previous adjacent hex entry detected for session  89  block  1
No previous adjacent hex entry detected for session  89  block  2
No previous adjacent hex entry detected for session  93  block  1
No previous adjacent hex entry detected for session  93  block  2
No previous adjacent hex entry detected for session  98  block  1
No previous adjacent hex entry detected for session  102  block  1
session  102  block  3
session  104  block  3
session  1  block  2
session  1  block  5
session  28  block  3
session  22  block  3
session  57  block  3
session  59  block  3
session  102  block  3
session  104  block  3
session  1  block  5
session  28  block  3
session  2

In [31]:
photrats.get_distanceToPort(getDistsFromDeadEnds=False)

# identify da change at hex discovery. add to df where other columns are distance to port and pRwd at port. then run regression of DA to distance and pRwd. If signal is a bonus - expected reward, makes sense that magnitude would scale inversely with val of approached port.

In [97]:
newHexDAchangeMeans,_ = calc_DaPeakDiffAfterNewPathInds(photrats,newHexAdjInds)

In [98]:
np.shape(newHexDAchangeMeans)

(10,)

In [124]:
newHexDaChanges = calc_DaPeakIndividualDiffsAfterNewPathInds(photrats,newHexAdjInds)

In [179]:
newHexRegDf = pd.DataFrame({"DA_change":newHexDaChanges,"dist2port":newDistsToPort,\
              "portVal":adjHexNextPortVal/100,'rat':photrats.df.loc[\
                                        newHexAdjInds,"rat"].astype(str).values})
newHexRegDf = newHexRegDf.loc[(newHexRegDf.notnull().all(axis=1)),:]

In [186]:
newHexRegDf.loc[:,"dist2port"] = valscale.fit_transform(\
        newHexRegDf.loc[:,"dist2port"].values.reshape(-1,1))
newHexRegDf.loc[:,"portVal"] = valscale.fit_transform(\
        newHexRegDf.loc[:,"portVal"].values.reshape(-1,1))

In [200]:
ratBetas = []
for rat in ratsWithSufficientNumbers:
    distFromNewDfRat = newHexRegDf.loc[newHexRegDf.rat==rat,:]
    y = distFromNewDfRat.loc[:,"DA_change"]
    X = distFromNewDfRat.loc[:,["dist2port","portVal"]]
    betas,pvals = run_smLinRegWithPval(X,y)
    ratBetas.append(betas)

/Users/timkrausz/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/timkrausz/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/timkrausz/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/timkrausz/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/timkrausz/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603

In [201]:
ratBetas

[array([ 4.2819347 , -0.2860685 , -3.43139157]),
 array([ 3.15090363,  0.44409047, -1.89022239]),
 array([ 4.63947051,  0.69144871, -1.0705713 ]),
 array([ 1.24829136,  4.17295315, -1.27010763]),
 array([  8.94329217, -12.39171072,  -3.0183711 ]),
 array([ 7.21973196, -5.51762129, -3.21643269]),
 array([ 2.28281531, -1.07706806,  0.21844859]),
 array([  3.64496033, -11.2918529 ,   4.44983239])]

In [234]:
fig = plt.figure()
sns.barplot(data=pd.DataFrame(ratBetas,columns=["icept","dist2port","pRwd"]),ci=95,color='grey',alpha=0.5)
sns.stripplot(data=pd.DataFrame(ratBetas,columns=["icept","dist2port","pRwd"]),\
              marker='D',color='deeppink',edgecolor='k',size=8,linewidth=3)
plt.ylabel(r"Regression $\beta \pm 95\% ci$ ")
plt.tight_layout()
fig.savefig(photrats.directory_prefix+"newHexDistAndProbBetas.pdf")

In [154]:
def run_smLinRegWithPval(X,y):
    X = np.hstack([np.ones(len(X)).reshape(-1,1),X])
    mod = sm.OLS(y, X).fit()
    coefs = mod.summary2().tables[1]['Coef.'].values
    pvals = mod.summary2().tables[1]["P>|t|"].values
    return coefs,pvals

In [155]:
y=newHexRegDf.loc[:,"DA_change"]
X = newHexRegDf.loc[:,["dist2port","portVal"]]
betas,pvals = run_smLinRegWithPval(X,y)

## betas for distance and port val are not significant. Should run regression for each animal that had enough data. Otherwise could do barplot and test if categories are different.

In [111]:
np.max(tracesPost[:,photrats.fs*1:],axis=1)

array([ 3.146  ,  5.     ,  1.124  ,  3.227  ,  0.5205 ,  5.992  ,
        0.5254 ,  4.54   ,  0.0371 ,  3.217  , -0.05737,  0.1938 ,
        0.4473 ,  0.6143 ,  3.06   ,  0.6396 ,  1.342  ,  1.486  ],
      dtype=float16)

# I was taking mean trace max - mean trace baseline, instead of mean of (max-baseline) for each trace

In [113]:
tracePeakRatMeans

[2.254, 5.824, 2.08, 4.2, 2.326, 4.516, 3.938, 1.992, 2.523, 2.266]

In [99]:
newHexDAchangeMeans

[1.442, 4.48, 0.889, 3.176, 1.177, 2.91, 3.246, 1.249, 1.642, 2.266]

In [86]:
photrats.set_plot_trace("green_z_scored")

In [618]:
availMeans,_ = calc_DaPeakDiffAfterNewPathInds(photrats,np.concatenate([enteredInds,ignoredInds]))

In [115]:
blockedMeans,missingInd = calc_DaPeakDiffAfterNewPathInds(photrats,blockedHexAdjInds)

In [116]:
blockedMeans

[1.214, 2.059, 0.6025, 0.8047, 2.543, 1.237, 2.438, 0.8555, 1.56, 1.081]

In [118]:
get_sigRatsPaired_from2samples(tracePeakRatMeans,blockedMeans,"greater")

p-value =  0.001953125


(False, True, True)

In [122]:
wilcoxon(tracePeakRatMeans)

WilcoxonResult(statistic=0.0, pvalue=0.001953125)

In [119]:
wilcoxon(blockedMeans)

WilcoxonResult(statistic=0.0, pvalue=0.001953125)

In [94]:
fig = plot_meanRatDaChangeAfterHexEntry(photrats,np.concatenate([enteredInds,ignoredInds]),blockedHexAdjInds,pltCol1="deeppink",pltCol2="k")
#plt.ylim(-.6,2.6)#-.5,2.3)
plt.xticks([0,1],["newly\navailable","newly\nblocked"])
plt.tight_layout()
fig.savefig(photrats.directory_prefix+"DaChangeNewlyAvailVblockedBarplot_firstDiscovery.pdf")
#fig.savefig(photrats.directory_prefix+photrats.plot_trace+"ChangeNewlyAvailVblockedBarplot_firstDiscovery.pdf")

blocked hex: 
avail hex: 
paired test
p-value =  0.0009765625


In [102]:
get_sigRats_fromMeanList(availMeans)

0.001953125


(False, True, True)

In [149]:
fig = plot_FirstEntryHexChange(photrats,np.concatenate([enteredInds,ignoredInds]),blockedHexAdjInds,legend_on=False)
#plt.ylim(-.9,2.2)
#fig.savefig(photrats.directory_prefix+"DaNewlyAvailVblocked_mean.pdf")
#fig.savefig(photrats.directory_prefix+photrats.plot_trace+"NewlyAvailVblocked_mean.pdf")

In [151]:
plt.ylabel("Mean running speed (cm/s)")
fig.savefig(photrats.directory_prefix+photrats.plot_trace+"NewlyAvailVblocked_mean.pdf")

- plot DA aligned to new hex discovery pooled by whether rat entered or ignored new path

In [29]:
photrats.set_plot_trace("green_z_scored")
fig = plotFirstAdjEntryByEnteredVsIgnored(photrats,enteredInds,ignoredInds,legend_on=True,pltCol1="#27aeef",pltCol2= "#b33dc6",ls2='-')
plt.ylim(-.9,2.2)
fig.savefig(photrats.directory_prefix+"DaEnteredVsIgnoredNewlyAvail_mean.pdf")

In [73]:
ignoredInclusionRats = np.array(list(ignored_n_PerRat.keys()))[np.where(np.array(list(ignored_n_PerRat.values()))>=3)]
inclusionInds = photrats.df.loc[photrats.df.rat.isin(ignoredInclusionRats)].index
ignoredInds2include = ignoredInds[np.where(np.isin(ignoredInds,inclusionInds))]

In [76]:
fig = plot_meanRatDaChangeAfterHexEntry(photrats,enteredInds,ignoredInds)
plt.xticks([0,1],["entered\navail","ignored\navail"])
plt.tight_layout()
fig.savefig(photrats.directory_prefix+"DaChangeEnteredVsIgnoredNewlyAvailBarplot.pdf")

blocked hex: 
p-value =  0.0546875
avail hex: 
p-value =  0.001953125
paired test
p-value =  0.23046875


- calculate difference in average rat DA trace (peak DA within 0.5s after hex entry - DA 1s before hex entry)

In [606]:
photrats.set_plot_trace("green_z_scored")

In [607]:
fig,availratmeans,avail_n_PerRat = plot_ratTracesAtHexChangeDiscovery(photrats,np.concatenate([enteredInds,ignoredInds]),pltCol="deeppink")
#fig.savefig(photrats.directory_prefix+"DaNewlyAvail_byRat.pdf")
fig,blockedratmeans,blocked_n_PerRat = plot_ratTracesAtHexChangeDiscovery(photrats,blockedHexAdjInds)
#fig.savefig(photrats.directory_prefix+"DaNewlyBlocked_byRat.pdf")
with open(photrats.directory_prefix+"availPathNumbers.txt", 'w') as f:
        f.write(str(avail_n_PerRat))
with open(photrats.directory_prefix+"blockedPathNumbers.txt", 'w') as f:
        f.write(str(blocked_n_PerRat))
fig = plotFirstEntryHexChangeMeanOverRats(photrats,availratmeans,blockedratmeans,ls2=':')
#fig.savefig(photrats.directory_prefix+"DaNewlyAvailvBlocked_byRat.pdf")


In [611]:
np.mean(availratmeans,axis=1)

array([0.0955 , 0.501  , 0.1608 , 0.2825 , 0.10706, 0.2976 , 0.6567 ,
       0.4949 , 0.2069 , 0.67   ], dtype=float16)

In [626]:
availMeans

[1.442, 4.48, 0.889, 3.176, 1.177, 2.91, 3.246, 1.249, 2.23, 1.72]

In [629]:
wilcoxon(availMeans,blockedMeans)#,alternative="greater")

WilcoxonResult(statistic=0.0, pvalue=0.001953125)

In [165]:
fig = plotFirstEntryHexChangeMeanOverRats(photrats,availratmeans,blockedratmeans,ls2=':')

In [174]:
ratsWithOkNum = [rat for rat in ignored_n_PerRat.keys() if ignored_n_PerRat[rat]>=3]

['IM-1272', 'IM-1276', 'IM-1292', 'IM-1322', 'IM-1478']

In [192]:
ignoredratmeans

[array([-0.3533 , -0.3564 , -0.3528 , ..., -0.087  , -0.0444 ,  0.03134],
       dtype=float16),
 array([ 0.2947 ,  0.225  ,  0.1694 , ..., -0.302  , -0.1936 , -0.09625],
       dtype=float16),
 array([0.153  , 0.1316 , 0.11334, ..., 0.00236, 0.04196, 0.07404],
       dtype=float16),
 array([-0.10345, -0.1081 , -0.1132 , ..., -0.2113 , -0.2081 , -0.2112 ],
       dtype=float16)]

In [195]:
fig,enteredratmeans,entered_n_PerRat = plot_ratTracesAtHexChangeDiscovery(photrats,enteredInds,pltCol="#27aeef")
fig.savefig(photrats.directory_prefix+"DaEnteredPath_byRat.pdf")
fig,ignoredratmeans,ignored_n_PerRat = plot_ratTracesAtHexChangeDiscovery(photrats,ignoredInds,pltCol="#b33dc6")
fig.savefig(photrats.directory_prefix+"DaIgnoredPath_byRat.pdf")
with open(photrats.directory_prefix+"enteredPathNumbers.txt", 'w') as f:
        f.write(str(entered_n_PerRat))
with open(photrats.directory_prefix+"ignoredPathNumbers.txt", 'w') as f:
        f.write(str(ignored_n_PerRat))
fig = plotFirstEntryHexChangeMeanOverRats(photrats,enteredratmeans,ignoredratmeans,pltCol1="#27aeef",pltCol2="#b33dc6")
fig.savefig(photrats.directory_prefix+"DaNewlyEnteredvIgnored_byRat.pdf")

/Users/timkrausz/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/timkrausz/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [198]:
entered_n_PerRat

{'IM-1272': 11,
 'IM-1273': 3,
 'IM-1276': 13,
 'IM-1291': 8,
 'IM-1292': 11,
 'IM-1322': 9,
 'IM-1398': 6,
 'IM-1434': 5,
 'IM-1478': 5,
 'IM-1532': 3}

In [197]:
np.sum([entered_n_PerRat[rat] for rat in entered_n_PerRat])

74

In [200]:
np.sum([ignored_n_PerRat[rat] for rat in ignored_n_PerRat if ignored_n_PerRat[rat]>2])

26

In [ ]:
np.where(ignored_n_PerRat>)

In [ ]:
fig = plotFirstEntryHexChangeMeanOverRats(photrats,enteredratmeans,ignoredratmeans,pltCol1="#27aeef",pltCol2="#b33dc6")

- CONTROL: plot avg da in all entries into newly available/blocked hex state on all other trials in session.

In [59]:
# identify the hex of first discovery

# identify all times during the same session when rat entered. (get indices)

# get traces aligned to these indices

# plot average response to same hexes

- instead, first plot DA at entry into all choice points.. then average response upon entry into all hexes (with distance from port >4)

In [63]:
del photrats.sesh_tmats

In [66]:
photrats.get_allChoicePoints()

## plot regression results

In [29]:
regCoefs = pd.read_csv(photrats.directory_prefix+chooseLregCoefsByRat.csv")
regSum = pd.read_csv(photrats.directory_prefix+chooseLregSummary.csv")

In [42]:
regSum

,Unnamed: 0,Estimate,Std. Error,z value,Pr(>|z|)
0,(Intercept),2.185913,0.333764,6.549282,5.781420e-11
1,pRwdScaled,1.685901,0.151973,11.093423,1.350207e-28
2,ldifScaled,-6.839847,0.595627,-11.483439,1.597954e-30


In [48]:
fig = plt.figure(figsize=(5,7))
xvals = ["Intercept","p(reward)","Distance"]
#sns.boxplot(data=regCoefs.loc[:,["pRwdScaled","ldifScaled"]],color='lightgrey')#,alpha=.5)
sns.barplot(data=regCoefs.loc[:,["(Intercept)","pRwdScaled","ldifScaled"]],color='lightgrey',ci=95)#,alpha=.5)
#plt.bar([0,1,2],regSum.Estimate.values)
sns.stripplot(data=regCoefs.loc[:,["(Intercept)","pRwdScaled","ldifScaled"]],color='k',size=8,marker='D',alpha=.9)
plt.axhline(y=0,ls='--',color='k')
plt.xticks([0,1,2],xvals)
#plotRegSigLevel(regSum,0,2.0)
#plotRegSigLevel(regSum,1,2.0)
#plt.ylim(-10,2.2)
plt.ylabel("Path choice ß value\n(mixed-effects)",fontsize='xx-large')
plt.tight_layout()

In [49]:
fig.savefig(photrats.directory_prefix+"mixedEffectsFullResults.pdf")

# plot single trial examples of maze configurations and tick plots.
- plot position over maze early vs late in block

In [26]:
from photometrySessionVizualizations import *

In [247]:
photrats.directory_prefix = loadpath


- create acceleration column from velocity

In [584]:
photrats.df.loc[photrats.df.vel.notnull(),"acc"] = photrats.df.loc[photrats.df.vel.notnull(),'vel'].diff()
photrats.df.loc[:,"acc"] = photrats.df.loc[:,"acc"].fillna(method="ffill")

In [583]:
plot_posOverlayAndTickPlot(photrats,s=99,posColor='cyan',edgCol='none',saveFig=True,\
                           secondHalf=True,plotOverlay=True,plotProbs=True,trans=0.1,vmin=1,vmax=8,density=False,blks=[1,2])

/Users/timkrausz/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maze-photometry-analyses/photometrySessionVizualizations.py:63: MatplotlibDeprecationWarning: Using a string of single character colors as a color sequence is deprecated since 3.2 and will be removed two minor releases later. Use an explicit list instead.
  plt.scatter(centdf.loc[barriers,0].values,centdf.loc[barriers,1].values,c=\
<ipython-input-545-0f8b219b588a>:67: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
  plt.tight_layout()
/Users/timkrausz/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maze-photometry-analyses/photometrySessionVizualizations.py:63: MatplotlibDeprecationWarning: Using a string of single character colors as a color sequence is deprecated since 3.2 and will be removed two minor releases later. Use an explicit list instead.
  plt.scatter(centdf.loc[barriers,0].values,centdf.loc[barriers,1].values,c=\
<ipyt

In [168]:
sesh = 27
#plot_posOverlayAndTickPlot(photrats,s=69,posColor='dodgerblue',edgCol='k',saveFig=True,secondHalf=False,plotOverlay=True,plotProbs=True)
plot_posOverlayAndTickPlot(photrats,s=sesh,posColor='cyan',edgCol='blue',saveFig=True,secondHalf=True,plotOverlay=True,plotProbs=True)
plot_posOverlayAndTickPlot(photrats,s=sesh,posColor='cyan',edgCol='blue',saveFig=True,secondHalf=False,plotOverlay=True,plotProbs=True)
#plot_posOverlayAndTickPlot(photrats,s=69,posColor='dodgerblue',edgCol='k',saveFig=True,secondHalf=True,plotOverlay=True,plotProbs=True)
#plot_posOverlayAndTickPlot(photrats,s=36,posColor='dodgerblue',edgCol='k',saveFig=True,secondHalf=True,plotOverlay=True,plotProbs=True)

<ipython-input-40-c2c15f210612>:105: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax3.set_yticklabels([''])
<ipython-input-40-c2c15f210612>:108: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax2.set_yticklabels([''])
<ipython-input-40-c2c15f210612>:111: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax3.set_yticklabels([''])
<ipython-input-40-c2c15f210612>:130: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
  plt.tight_layout()
<ipython-input-40-c2c15f210612>:63: MatplotlibDeprecationWarning: Using a string of single character colors as a color sequence is deprecated since 3.2 and will be removed two minor releases later. Use an explicit list instead.
  plt.scatter(centdf.loc[barriers,0].values,centdf.loc[barriers,1].values,c=\
<ipython-input-40-c2c15f210612>:160: UserWarning: Tight layout not applied. tight_layout ca

In [91]:
plt.close("all")

In [595]:
sesh = 90
blocks = [2,3]
plot_10tri_posOverlay(photrats,s=sesh,posColor='midnightblue',edgCol='none',saveFig=True,groupOfTen=1,plotOverlay=True,plotProbs=True,blks=blocks)
plot_10tri_posOverlay(photrats,s=sesh,posColor='midnightblue',edgCol='none',saveFig=True,groupOfTen=2,plotOverlay=True,plotProbs=True,blks=blocks)
plot_10tri_posOverlay(photrats,s=sesh,posColor='midnightblue',edgCol='none',saveFig=True,groupOfTen=3,plotOverlay=True,plotProbs=True,blks=blocks)
#plot_10tri_posOverlay(photrats,s=sesh,posColor='cyan',edgCol='blue',saveFig=True,groupOfTen=4,plotOverlay=True,plotProbs=True)

/Users/timkrausz/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maze-photometry-analyses/photometrySessionVizualizations.py:63: MatplotlibDeprecationWarning: Using a string of single character colors as a color sequence is deprecated since 3.2 and will be removed two minor releases later. Use an explicit list instead.
  plt.scatter(centdf.loc[barriers,0].values,centdf.loc[barriers,1].values,c=\
<ipython-input-593-aa70ba886096>:28: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
  plt.tight_layout()
/Users/timkrausz/Library/Mobile Documents/com~apple~CloudDocs/Documents/Maze-photometry-analyses/photometrySessionVizualizations.py:63: MatplotlibDeprecationWarning: Using a string of single character colors as a color sequence is deprecated since 3.2 and will be removed two minor releases later. Use an explicit list instead.
  plt.scatter(centdf.loc[barriers,0].values,centdf.loc[barriers,1].values,c=\
<ipyt

In [89]:
for sesh in photrats.df.loc[photrats.df.session_type=="barrier","session"].unique()[:10]:
    plot_10tri_posOverlayAndTickPlot(photrats,s=sesh,posColor='cyan',edgCol='blue',saveFig=True,secondTen=False,plotOverlay=True,plotProbs=True)
    plot_10tri_posOverlayAndTickPlot(photrats,s=sesh,posColor='cyan',edgCol='blue',saveFig=True,secondTen=True,plotOverlay=True,plotProbs=True)

/Users/timkrausz/Documents/Maze-photometry-analyses/photometrySessionVizualizations.py:130: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
  plt.tight_layout()
/Users/timkrausz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
/Users/timkrausz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
/Users/timkrausz/Documents/Maze-photometry-analyses/photometrySessionVizualizations.py:130: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
  plt.tight_layout()
/Users/timkrausz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Tight layout not ap